In [1]:
"""Accuracy 97%"""
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True) # one_hot True면 Y값을 one_hot으로 읽어옴

nb_classes = 10

# input place holders
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

"""
아까는 3단이였는데, 5단으로 늘리고,
아까는 256 output을 512로 두배로 늘리자.

근데 accuracy가 97% 별로 변화가 없다.. 왜??
뭐 데이터에 따라 다른 요인이 있을텐데, 아마 overfitting이 있었을 것이다.
학습데이터를 너무 잘 기억을 해버려서 새로운 테스팅 데이터가 들어왔을때 결과가 나빠지는 overfitting.

이걸 막기 위한 방법중 하나가 바로 dropout
"""
W1 = tf.get_variable("W1", shape=[784, 512],
                    initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1) # Activation function으로 relu 사용.

W2 = tf.get_variable("W2", shape=[512, 512],
                    initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.get_variable("W3", shape=[512, 512],
                    initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([512]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

W4 = tf.get_variable("W4", shape=[512, 512],
                  initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([512]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)

W5 = tf.get_variable("W5", shape=[512, 10],
                  initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5



# define cost/loss & optimizer
learning_rate = 0.001
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


# Test model
is_correct = tf.equal(tf.arg_max(hypothesis, 1), tf.arg_max(Y, 1)) # one_hot인 Y
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

# parameters
# 데이터가 크니깐 한꺼번에 할수없으니 조금씩 학습
# epoch: 몇갠지는 모르지만 전체 데이터셋을 한번 다 학습시킨것을 1 epoch
# batch_size: 전체 데이터를 한꺼번에 메모리에 못올릴때 짤라서 올리는데 그 size
# iterations: 전체 데이터가 1000개이고 batch_size가 500개 일때 1 epoch을 하기 위해선 2 iterations을 돌아야한다.
training_epochs = 15
batch_size = 100

with tf.Session() as sess: # 세션 열고
    # Initialize Tensorflow variables
    sess.run(tf.global_variables_initializer())
    # Training cycle
    for epoch in range(training_epochs): # 15번 돌아가겠지
        avg_cost = 0
        # 전체 사이즈를 batch_size로 나누면 iterations 수 구할 수있음
        total_batch = int(mnist.train.num_examples / batch_size)
        
        for i in range(total_batch): # i가 iterations
            batch_xs, batch_ys = mnist.train.next_batch(batch_size) # 100개씩 읽어와서 학습을 함
            c, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y: batch_ys})
            avg_cost += c / total_batch
        
        print('Epoch: ', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    
    # sess.run()으로 돌릴 수도 있고, accuracy.eval()로도 돌릴 수 있음
    print("Accuracy: ", accuracy.eval(session=sess,
                                 feed_dict={X: mnist.test.images, Y: mnist.test.labels}))
    

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Epoch:  0001 cost = 0.319509954
Epoch:  0002 cost = 0.105553514
Epoch:  0003 cost = 0.074446936
Epoch:  0004 cost = 0.054454666
Epoch:  0005 cost = 0.044734532
Epoch:  0006 cost = 0.036522552
Epoch:  0007 cost = 0.030568030
Epoch:  0008 cost = 0.026970153
Epoch:  0009 cost = 0.026073136
Epoch:  0010 cost = 0.022817053
Epoch:  0011 cost = 0.020039665
Epoch:  0012 cost = 0.017611237
Epoch:  0013 cost = 0.017844869
Epoch:  0014 cost = 0.017029033
Epoch:  0015 cost = 0.014028624
Accuracy:  0.9784
